In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
FILE_PATH = os.path.join(os.path.abspath('.'))
BASE_DIR = os.path.dirname(FILE_PATH)

In [ ]:
df_train = pd.read_csv(os.path.join(FILE_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(FILE_PATH, 'test.csv'))

In [ ]:
## Dataset apenas com o target (variável alvo) preenchida
df_train_modified = df_train[df_train['NU_NOTA_MT'].isnull() == False]

In [ ]:
## Seleção de features
df_auxiliar = pd.DataFrame(data={'Coluna': df_train_modified.corr()['NU_NOTA_MT'].index, 
                                'Valores': df_train_modified.corr()['NU_NOTA_MT'].values})

In [ ]:
df_auxiliar = df_auxiliar[df_auxiliar['Coluna'] != 'NU_NOTA_MT'].copy() ## Retirar target do problema
df_auxiliar.dropna(inplace=True) ## Retirar linhas que não possuem correlação com target

## Valores de 0.25 e -0.20 escolhidos referentes a uma correlação média positiva e negativa com NU_NOTA_MT
df_auxiliar = df_auxiliar[(df_auxiliar['Valores'] > 0.25) | (df_auxiliar['Valores'] < -0.20)].copy()
df_auxiliar.reset_index(drop = True, inplace=True)

In [ ]:
## Features correlacioandas com o target
features_corr = df_auxiliar['Coluna'].tolist()

## Verificação se as features selecionadas existem no df_test
features = []
for i in features_corr:
    if i in df_test.columns.tolist():
        features.append(i)
    else:
        print("Features {} não existente no df_test".format(i))
print(features)

In [ ]:
target = 'NU_NOTA_MT' ## Definição do target do problema

In [ ]:
## Tratamento dos datasets
df_train_modified = df_train_modified[features+[target]].copy()
df_train_modified.fillna(0, inplace=True)

df_test_modified = df_test[features].copy()
df_test_modified.fillna(0, inplace=True)

In [ ]:
## Divisão das bases de treinamento e teste
X_train = df_train_modified[features]
X_test = df_test_modified[features]
y_train = df_train_modified[target]

In [ ]:
## Padronização das variáveis
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
x_train = sc.fit_transform(X_train)  
x_test = sc.transform(X_test)

In [ ]:
## Busca pelos melhores parametros
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()
param_grid = { 
        "n_estimators" : [100, 200, 300, 400, 500],
        "max_depth" : [2, 4, 6, 8, 10],
        "max_features" : ["auto", "sqrt", "log2"],
        "min_samples_split" : [20, 40, 60, 80, 100],
        "bootstrap": [True, False],
        }

grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

grid.fit(X_train, y_train)

grid.best_score_ , grid.best_params_

In [ ]:
## Utilização dos melhores parametros encontrados
estimator = RandomForestRegressor(
                                max_depth=8,
                                n_estimators=500,
                                 )
estimator.fit(X_train, y_train)
estimator.predict(X_test)

In [ ]:
answer = pd.DataFrame(data={'NU_INSCRICAO':df_test['NU_INSCRICAO'],
                  'NU_NOTA_MT':estimator.predict(df_test_modified)})
answer.to_csv(os.path.join(FILE_PATH, 'answer.csv'), sep=',', index=False)